# 웹 스크래핑(web scraping with python)
    어떠한 웹페이지의 HTML 내용을 분석하여 원하는 값을 추출해내는 것을 스크래핑(scraping)이라고 합니다.
    python의 기본패키지 및 BeautifulSoup을 이용해 daum.net의 검색결과 페이지를 스크래핑하는 예제 코드를 첨부합니다.

    이 코드를 실행하려면 먼저 BeautifulSoup 패키지를 설치해야 합니다.
    BeautifulSoup 패키지의 이름은 간단하게 'bs4' 입니다.

    그래서 윈도우 커맨드 창에서 'easy_install bs4' 라고 실행하면 패키지를 설치할 수 있습니다.
    그런데 easy_install 를 설치하지 않았다면 설치해야 합니다.
    
    
    bs4 설치방법
    1) easy_install을 설치한다 방법은 http://blog.colab.kr/11 참고.
    2) easy_install 이 설치된 후에 윈도우 command 창에서 다음과 같이 실행한다.
    $ easy_install bs4

In [17]:
# coding: utf-8
import urllib2
import urllib
import urlparse
import time
from bs4 import BeautifulSoup as bs


USER_AGENT_CHROME = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_6_8) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.37 Safari/537.36"
USER_AGENT_FIREFOX = "Mozilla/5.0 (X11; U; FreeBSD i386; en-US; rv:1.7.8) Gecko/20050609 Firefox/1.0.4"
AGENTS = [USER_AGENT_FIREFOX, USER_AGENT_CHROME]
agent_cnt = 0

ImportError: No module named 'urllib2'

In [16]:
def fetch(web_url):
  '''
  주어진 웹페이지의 HTML 내용을 읽어옴
  :param web_url: the url of a webpage
  :return: HTML contents (문자열)
  '''
  global agent_cnt
  agent_cnt += 1
  try:
    req = urllib.Request(web_url)
    req.add_header("User-Agent", AGENTS[agent_cnt % 2])
    req.add_header("Connection", "Keep-Alive")
    #print req.headers
    f = urllib.urlopen(req)
    html = f.read()
    #print len(html)
    return html
  except urllib.HTTPError, e:
    print e.code
    print e.msg
    return ''


SyntaxError: invalid syntax (<ipython-input-16-0e72e9b9719c>, line 18)